# Движение карты

Начнем с загрузки фона. тут все просто)

In [ ]:
from first_engine.game import Game
from first_engine import objects
import pygame
from pprint import pprint


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=8)
        hero.load_sprites(name='stand_right', path='./sprites/viking/stand/right/')
        hero.load_sprites(name='stand_left', path='./sprites/viking/stand/left/')
        hero.load_sprites(name='run_left', path='./sprites/viking/run/left/')
        hero.load_sprites(name='run_right', path='./sprites/viking/run/right/')
        
        # Опишем пару стен
        wall = objects.Barrier(self.surface, objects=[hero], width=200, height=50, x=300, y=400)
        wall2 = objects.Barrier(self.surface, objects=[hero], width=200, height=300, x=600, y=300)

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(FPS=60)
                wall.blit()
                wall2.blit()

                # Объекты
                hero.blit()
                hero.play_animation()
                hero.drop(speed_up=1, max_speed=20)
                left = hero.motion_left()
                right = hero.motion_right()

                if hero.body.x > 500: # Если герой дошол примерно до середины экрана
                    hero.body.x = 500 # тогда он перестает двигаться
                    # а начинают двигаться окружающие объекты
                    wall.body.x -= right
                    wall2.body.x -= right

                wall.resistance()
                wall2.resistance()
                self.window_borders([hero])

                hero.action_jump()

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=800, height=600).run()

Теперь попробуем загрузить фон

In [ ]:
...
while self.RUNNER:
    if not game_over:
        self.cycle_init(FPS=60)
        self.surface.blit(pygame.image.load('./forest.jpg'), dest=(0, -100))
...

Обратите внимания, что наблюдаются просадки ФПС это потому, что мы на каждой итерации заново загружаем фон, исправим это.

In [ ]:
...
bg = pygame.image.load('./forest.jpg') # загрузим картинку заранее
bg_body = bg.get_rect(topleft=(0, -100)) # сразу вытащим её тело и размерим по координатам по отношению к экрану

while self.RUNNER:
    if not game_over:
        self.cycle_init(FPS=60)
        self.surface.blit(bg, dest=bg_body) # размещаем в экране
...

Давайте долбавим эффект при котором задний фон тоже будет двигаться

In [ ]:
...
if hero.body.right > 500:
    hero.body.right = 500
    wall.body.x -= right
    wall2.body.x -= right
    bg_body.x -= right / 2 # задний фон будет двигаться с половиной от скорости персонажа
...

Здесь фон будет слева отслаиваться от экрана, давайте это исправим

In [ ]:
...
if hero.body.right > 500 and bg_body.right > 800: # добавим ограничение справа
    hero.body.right = 500
    wall.body.x -= right
    wall2.body.x -= right
    bg_body.x -= right / 2
...

Тепер добавим движение объектов слева

In [ ]:
...
if hero.body.right > 500 and bg_body.right > 800:
    hero.body.right = 500
    wall.body.x -= right
    wall2.body.x -= right
    bg_body.x -= right / 2
elif hero.body.left < 300 and bg_body.left < 0: # все тоже самое что и при движении вправо но в обратную сторону
    hero.body.left = 300
    wall.body.x += left
    wall2.body.x += left
    bg_body.x += left / 2
...

# Задание

- Создайте еще персонажей, которые будут врагами
- придумайте любой спосод атаки для всех персонажей
- Реализуйте потерю здоровья и смерть персонажей